In [ ]:
import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import xml.etree.ElementTree as ET
import zipfile

## unzip the exported dataset from CVAT  
set zipfile name as CVAT dataset

In [ ]:
# Parameters Setting

zipfile_name = '1stdataset20230420'

# ==================================================

with zipfile.ZipFile(f'{zipfile_name}.zip', 'r') as zf:
    if not os.path.exists(f'{zipfile_name}'):
        os.makedirs(f'{zipfile_name}')

    zf.extractall(f'{zipfile_name}')

## Label Box Extraction
`def image_names_parser(path)` -> return all image names from the annotations
`def labels_parser(path, image_name)` -> return label boxs of corresponding image

test: check if the box is in the right place


In [ ]:
def image_names_parser(path):
    tree = ET.parse(f'{path}/annotations.xml')
    root = tree.getroot()
    image_names = []
    for image in root.findall('image'):
        image_names.append(image.get('name'))
    return image_names

def labels_parser(path, image_name):
    tree = ET.parse(f'{path}/annotations.xml')
    root = tree.getroot()

    labels = {}
    for i, label in enumerate(root.iter('label')):
        labels[label.find('name').text] = i

    image_node = None
    for image in root.findall('image'):
        if image.get('name') == image_name:
            image_node = image

    boxs = []
    coordinates = ['ytl', 'xtl', 'ybr', 'xbr']
    for i in range(len(labels)):
        boxs.append([])
    for box in image_node.findall('box'):
        points = []
        for coordinate in coordinates:
            points.append(int(box.get(coordinate).split(".")[0]))
        boxs[labels[box.get('label')]].append(points)

    return boxs
boxs = labels_parser(zipfile_name, 'WLT_350_210926 P_2111131 WLT_350_210926__002 P_2111131_processed.JPG')
for i in range(len(boxs)):
    print(boxs[i])

## Check if the annotations is shown as they are on CVAT
set the index of images and the feature type in the CVAT annotations

Flag of the feature type
- small strong hyperbola: 0
- small weak hyperbola: 1
- noise: 2
- large strong hyperbola: 3
- large weak hyberbola: 4
- no feature: 5

In [ ]:
# Parameters Setting

index = 0
flag = 0

# ==================================================

test_image = image_names_parser(zipfile_name)[index]
boxs = labels_parser(zipfile_name, test_image)

print(test_image)
img = cv2.imread(f'processed/{test_image}')
color = np.array([3, 67, 223])
box_width = 3
for box in boxs[flag]:
    img[box[0] - box_width:box[0] + box_width, box[1]:box[3], :] = np.ones_like(img[box[0] - box_width:box[0] + box_width, box[1]:box[3], :]) * color
    img[box[2] - box_width:box[2] + box_width, box[1]:box[3], :] = np.ones_like(img[box[2] - box_width:box[2] + box_width, box[1]:box[3], :]) * color
    img[box[0]:box[2], box[1] - box_width:box[1] + box_width, :] = np.ones_like(img[box[0]:box[2], box[1] - box_width:box[1] + box_width, :]) * color
    img[box[0]:box[2], box[3] - box_width:box[3] + box_width, :] = np.ones_like(img[box[0]:box[2], box[3] - box_width:box[3] + box_width, :]) * color

plt.imshow(img)
plt.show()

In [ ]:
def check_out_of_box(sliding_window, box):
    if sliding_window[2] < box[0] or sliding_window[0] > box[2]:
        return True
    if sliding_window[3] < box[1] or sliding_window[1] > box[3]:
        return True
    return False

def check_box(sliding_window, box, overlapping):
    if check_out_of_box(sliding_window, box):
        return False
    
    box_area = (box[2] - box[0]) * (box[3] - box[1])
    overlap_area = (min(sliding_window[2], box[2]) - max(sliding_window[0], box[0])) * (min(sliding_window[3], box[3]) - max(sliding_window[1], box[1]))
    if overlap_area / box_area < overlapping:
        return False    
    
    return True

In [ ]:
def crop_image(save_path, zipfile_name, image_index, image_name, boxs, overlapping, sliding_window_size=200, step=40):
    img = cv2.imread(f'{save_path}/{image_name}')
    feature_types = ['small strong hyperbola', 'small weak hyperbola', 'noise', 'large strong hyperbola', 'large weak hyberbola', 'no feature']
    # exclude the left axis and data above 0 ms
    # Boundary constant of images
    IMAGE_START = 64
    AXIS_ZERO = 121
    image_shape = img.shape
    image_start = [AXIS_ZERO, IMAGE_START]
    print('image shape:', image_shape)
    height = math.ceil((image_shape[0] - sliding_window_size - image_start[0]) / step)
    width = math.ceil((image_shape[1] - sliding_window_size - image_start[1]) / step)

    no_feature_index = len(feature_types) - 1
    cropped_image_type = np.ones((height, width)) * no_feature_index
    for k, box_list in enumerate(boxs):
        for box in box_list:
            box_index = [(b - image_start[0]) / step for b in box]
            start = [math.floor(box_index[0]), math.floor(box_index[1])]
            end = [math.ceil(box_index[2]), math.ceil(box_index[3])]
            sub_height_start = int(math.floor((AXIS_ZERO - image_start[0]) / step))
            sub_width_start = int(math.floor((IMAGE_START - image_start[1]) / step))
            for j in range(max(start[0] - int(sliding_window_size/step), sub_height_start), min(end[0] + int(sliding_window_size/step), height)):
                for i in range(max(start[1] - int(sliding_window_size/step), sub_width_start), min(end[1] + int(sliding_window_size/step), width)):
                    if cropped_image_type[j, i] == no_feature_index and check_box([j, i, j + int(sliding_window_size/step), i + int(sliding_window_size/step)], box_index, overlapping):
                        cropped_image_type[j, i] = k
    
    # create folders for cropped images
    for i in range(len(feature_types)):
        if not os.path.exists(f'{save_path}/cropped/{sliding_window_size}_{step}/{zipfile_name}/{image_index}/{i}'):
            os.makedirs(f'{save_path}/cropped/{sliding_window_size}_{step}/{zipfile_name}/{image_index}/{i}')
    # output cropped images
    for j in range(height):
        count = [0] * len(feature_types)
        for k in range(len(count)):
            for path in os.listdir(f'{save_path}/cropped/{sliding_window_size}_{step}/{zipfile_name}/{image_index}/{k}'):
                if path.startswith(f'{j * step}_'):
                    image_num = int(path.split(".jpg")[0].split("_")[1])
                    count[k] = max(count[k], image_num)

        for i in range(width):
            sliding_window = [AXIS_ZERO + j * step, IMAGE_START + i * step, AXIS_ZERO + sliding_window_size + j * step, IMAGE_START + sliding_window_size + i * step]
            sub_img = img[sliding_window[0]:sliding_window[2], sliding_window[1]:sliding_window[3]]
            feature_type = int(cropped_image_type[j, i])
            cv2.imwrite(f'{save_path}/cropped/{sliding_window_size}_{step}/{zipfile_name}/{image_index}/{feature_type}/{j * step}_{count[feature_type]}.jpg', sub_img)
            count[feature_type] += 1

    print(f'Finished generating images')

In [ ]:
# Parameters Setting

IMAGE_START = 64
AXIS_ZERO = 121

img = cv2.imread(f'processed/{test_image}')
box_width = 3
image_shape = img.shape
image_start = [AXIS_ZERO, IMAGE_START]
sliding_window_size = 200
step = 40

height = math.ceil((image_shape[0] - sliding_window_size - image_start[0]) / step)
width = math.ceil((image_shape[1] - sliding_window_size - image_start[1]) / step)

j, i = 0, 0
sliding_window = [AXIS_ZERO + j * step, IMAGE_START + i * step, AXIS_ZERO + sliding_window_size + j * step, IMAGE_START + sliding_window_size + i * step]
sub_img = img[sliding_window[0]:sliding_window[2], sliding_window[1]:sliding_window[3]]
            
plt.imshow(sub_img)
plt.show()

## Set the save path of cropped images
The cropped image save in the processed directory in default.

### Parameter
- save_path: default in 'processed'
- flag: feature type. If all kind of feature is required, run through flag 0-5
- sliding_window_size: size of the cropped images. 200 x 200 pixels in default
- step: step of sliding window. 100 pixels in default
- overlapping: acceptance rate of overlapping of sliding windows and feature box. 0.8 in default, which means that sliding window covers above 80% of feature box

Flag of the feature type
- small strong hyperbola: 0
- small weak hyperbola: 1
- noise: 2
- large strong hyperbola: 3
- large weak hyberbola: 4
- no feature: 5

In [ ]:
# Parameters Setting

save_path = 'unprocessed_images'
sliding_window_size = 200
step = 40
overlapping = 0.8

# ==================================================

image_names = image_names_parser(zipfile_name)
for image_index, image_name in enumerate(image_names):
    boxs = labels_parser(zipfile_name, image_name)
    image_name = image_name.replace('_processed', '')
    print(f'cropping {image_name} ...')
    crop_image(save_path, zipfile_name, image_index, image_name, boxs, overlapping, sliding_window_size=sliding_window_size, step=step)